# Imports and Helper Functions

In [ ]:
import multiprocessing
num_available_cpus = multiprocessing.cpu_count()

print("Number of available CPUs:", num_available_cpus)

import sys

import math
import time
import tqdm

import numpy as np
import scipy as sp
from scipy import stats
from scipy.spatial import ConvexHull

import itertools
import logging
import matplotlib.pyplot as plt

import pandas as pd
import h5py

from sklearn import metrics

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
import torch.utils.data as utils

from argparse import ArgumentParser
import re

sys.path.append("../new_flows")
from flows import RealNVP, Planar, MAF
from models import NormalizingFlowModel

In [ ]:
from nflows.flows.base import Flow
from nflows.flows.autoregressive import MaskedAutoregressiveFlow
from nflows.distributions.normal import StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform, MaskedPiecewiseQuadraticAutoregressiveTransform, MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation

In [ ]:
from helper_functions import *

In [ ]:
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device =", device)
torch.set_default_tensor_type('torch.cuda.FloatTensor') if torch.cuda.is_available() else print ('cpu')

torch.set_num_threads(num_available_cpus)

print("Number of threads:", torch.get_num_threads())
print("Number of interop threads:", torch.get_num_interop_threads())

# Load models

In [ ]:
num_features = 14
hidden_features = 56

num_layers = 4
num_blocks_per_layer = 4
#num_iter = 10000
num_iter = 1000
print_interval = 20

#Current flow_type options: 'MAF', 'NSQUAD' (neural spline quadratic), 'NSRATQUAD' (neural spline rational quadratic)
flow_type = 'NSQUAD'

In [ ]:
filename = 'Pure_NF_%s_k%s_hf%s_nbpl%s' % (flow_type, num_layers, hidden_features, num_blocks_per_layer)
bkg_model = torch.load("pure_flows/bkg_%s.pt" % (filename))
sig_model = torch.load("pure_flows/sig_%s.pt" % (filename))

# Load and Process Samples

In [ ]:
num_bkg_batches = 1
num_batches = 2    # for everything that isn't QCD background

In [ ]:
bkg_data, bkg_unnorm_data, bkg_masses = LAPS_test(sample_type = 'qcdbkg', num_batches = num_bkg_batches)
print(bkg_data.shape)
bkgtr_bkg_losses = -bkg_model.log_prob(bkg_data)[0].detach().cpu().numpy()
sigtr_bkg_losses = -sig_model.log_prob(bkg_data)[0].detach().cpu().numpy()

In [ ]:
sig_data, sig_unnorm_data, sig_masses = LAPS_test(sample_type = 'wprimesig', num_batches = num_batches)
print(sig_data.shape)
bkgtr_sig_losses = -bkg_model.log_prob(sig_data)[0].detach().cpu().numpy()
sigtr_sig_losses = -sig_model.log_prob(sig_data)[0].detach().cpu().numpy()

In [ ]:
graviton_data, graviton_unnorm_data, graviton_masses = LAPS_test(sample_type = 'graviton', num_batches = num_batches)
print(graviton_data.shape)
bkgtr_graviton_losses = -bkg_model.log_prob(graviton_data)[0].detach().cpu().numpy()
sigtr_graviton_losses = -sig_model.log_prob(graviton_data)[0].detach().cpu().numpy()

In [ ]:
Wkk_data, Wkk_unnorm_data, Wkk_masses = LAPS_test(sample_type = 'Wkk', num_batches = num_batches)
print(Wkk_data.shape)
bkgtr_Wkk_losses = -bkg_model.log_prob(Wkk_data)[0].detach().cpu().numpy()
sigtr_Wkk_losses = -sig_model.log_prob(Wkk_data)[0].detach().cpu().numpy()

In [ ]:
Wp2000_data, Wp2000_unnorm_data, Wp2000_masses = LAPS_test(sample_type = 'Wp2000', num_batches = num_batches)

Wp2000_num_samples = Wp2000_data.shape[0]
sampling_indices = np.random.randint(Wp2000_num_samples, size = int(0.05 * Wp2000_num_samples))
Wp2000_data = Wp2000_data[sampling_indices, :]
Wp2000_unnorm_data = Wp2000_unnorm_data[sampling_indices, :]
Wp2000_masses = Wp2000_masses[sampling_indices, :]

print(Wp2000_data.shape)
bkgtr_Wp2000_losses = -bkg_model.log_prob(Wp2000_data)[0].detach().cpu().numpy()
sigtr_Wp2000_losses = -sig_model.log_prob(Wp2000_data)[0].detach().cpu().numpy()

In [ ]:
Wp3000_data, Wp3000_unnorm_data, Wp3000_masses = LAPS_test(sample_type = 'Wp3000', num_batches = num_batches)

Wp3000_num_samples = Wp3000_data.shape[0]
sampling_indices = np.random.randint(Wp3000_num_samples, size = int(0.05 * Wp3000_num_samples))
Wp3000_data = Wp3000_data[sampling_indices, :]
Wp3000_unnorm_data = Wp3000_unnorm_data[sampling_indices, :]
Wp3000_masses = Wp3000_masses[sampling_indices, :]

print(Wp3000_data.shape)
bkgtr_Wp3000_losses = -bkg_model.log_prob(Wp3000_data)[0].detach().cpu().numpy()
sigtr_Wp3000_losses = -sig_model.log_prob(Wp3000_data)[0].detach().cpu().numpy()

In [ ]:
Wp5000_data, Wp5000_unnorm_data, Wp5000_masses = LAPS_test(sample_type = 'Wp5000', num_batches = num_batches)

Wp5000_num_samples = Wp5000_data.shape[0]
sampling_indices = np.random.randint(Wp5000_num_samples, size = int(0.05 * Wp5000_num_samples))
Wp5000_data = Wp5000_data[sampling_indices, :]
Wp5000_unnorm_data = Wp5000_unnorm_data[sampling_indices, :]
Wp5000_masses = Wp5000_masses[sampling_indices, :]

print(Wp5000_data.shape)
bkgtr_Wp5000_losses = -bkg_model.log_prob(Wp5000_data)[0].detach().cpu().numpy()
sigtr_Wp5000_losses = -sig_model.log_prob(Wp5000_data)[0].detach().cpu().numpy()

In [ ]:
RSGraviton2000_data, RSGraviton2000_unnorm_data, RSGraviton2000_masses = LAPS_test(sample_type = 'RSGraviton2000', num_batches = num_batches)

RSGraviton2000_num_samples = RSGraviton2000_data.shape[0]
sampling_indices = np.random.randint(RSGraviton2000_num_samples, size = int(0.05 * RSGraviton2000_num_samples))
RSGraviton2000_data = RSGraviton2000_data[sampling_indices, :]
RSGraviton2000_unnorm_data = RSGraviton2000_unnorm_data[sampling_indices, :]
RSGraviton2000_masses = RSGraviton2000_masses[sampling_indices, :]

print(RSGraviton2000_data.shape)
bkgtr_RSGraviton2000_losses = -bkg_model.log_prob(RSGraviton2000_data)[0].detach().cpu().numpy()
sigtr_RSGraviton2000_losses = -sig_model.log_prob(RSGraviton2000_data)[0].detach().cpu().numpy()

In [ ]:
Qstar2000_data, Qstar2000_unnorm_data, Qstar2000_masses = LAPS_test(sample_type = 'Qstar2000', num_batches = num_batches)

Qstar2000_num_samples = Qstar2000_data.shape[0]
sampling_indices = np.random.randint(Qstar2000_num_samples, size = int(0.05 * Qstar2000_num_samples))
Qstar2000_data = Qstar2000_data[sampling_indices, :]
Qstar2000_unnorm_data = Qstar2000_unnorm_data[sampling_indices, :]
Qstar2000_masses = Qstar2000_masses[sampling_indices, :]

print(Qstar2000_data.shape)
bkgtr_Qstar2000_losses = -bkg_model.log_prob(Qstar2000_data)[0].detach().cpu().numpy()
sigtr_Qstar2000_losses = -sig_model.log_prob(Qstar2000_data)[0].detach().cpu().numpy()

In [ ]:
Wkk2000_data, Wkk2000_unnorm_data, Wkk2000_masses = LAPS_test(sample_type = 'Wkk2000', num_batches = num_batches)

Wkk2000_num_samples = Wkk2000_data.shape[0]
sampling_indices = np.random.randint(Wkk2000_num_samples, size = int(0.05 * Wkk2000_num_samples))
Wkk2000_data = Wkk2000_data[sampling_indices, :]
Wkk2000_unnorm_data = Wkk2000_unnorm_data[sampling_indices, :]
Wkk2000_masses = Wkk2000_masses[sampling_indices, :]

print(Wkk2000_data.shape)
bkgtr_Wkk2000_losses = -bkg_model.log_prob(Wkk2000_data)[0].detach().cpu().numpy()
sigtr_Wkk2000_losses = -sig_model.log_prob(Wkk2000_data)[0].detach().cpu().numpy()

# Master QUAK Space

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
plt.scatter(bkgtr_bkg_losses, sigtr_bkg_losses, s=2, label = 'Bkg samples')
plt.scatter(bkgtr_sig_losses, sigtr_sig_losses, s=2, label = 'W\' sig samples')
#plt.scatter(bkgtr_graviton_losses, sigtr_graviton_losses, s=2, label = 'Graviton sig samples')
#plt.scatter(bkgtr_Wkk_losses, sigtr_Wkk_losses, s=2, label = 'Wkk sig samples')
plt.scatter(bkgtr_Wp2000_losses, sigtr_Wp2000_losses, s=2, label = 'Wp2000 sig samples')
plt.scatter(bkgtr_Wp3000_losses, sigtr_Wp3000_losses, s=2, label = 'Wp3000 sig samples')
plt.scatter(bkgtr_Wp5000_losses, sigtr_Wp5000_losses, s=2, label = 'Wp5000 sig samples')
plt.xlim(0,100)
plt.ylim(0,100)
plt.xlabel("BKG-Trained Model Loss")
plt.ylabel("SIG-Trained Model Loss")
plt.title("Synthetic Testing Data QUAK Space")
plt.legend()
plt.show()

In [ ]:
plt.scatter(bkgtr_bkg_losses, sigtr_bkg_losses, s=2, label = 'Bkg samples')
plt.scatter(bkgtr_sig_losses, sigtr_sig_losses, s=2, label = 'W\' sig samples')
#plt.scatter(bkgtr_graviton_losses, sigtr_graviton_losses, s=2, label = 'Graviton sig samples')
#plt.scatter(bkgtr_Wkk_losses, sigtr_Wkk_losses, s=2, label = 'Wkk sig samples')
plt.scatter(bkgtr_RSGraviton2000_losses, sigtr_RSGraviton2000_losses, s=2, label = 'RSGraviton2000 sig samples')
plt.scatter(bkgtr_Qstar2000_losses, sigtr_Qstar2000_losses, s=2, label = 'Qstar2000 sig samples')
plt.scatter(bkgtr_Wp5000_losses, sigtr_Wp5000_losses, s=2, label = 'Wp2000 sig samples')
plt.scatter(bkgtr_Wkk2000_losses, sigtr_Wkk2000_losses, s=2, label = 'Wkk2000 sig samples')
plt.xlim(0,100)
plt.ylim(0,100)
plt.xlabel("BKG-Trained Model Loss")
plt.ylabel("SIG-Trained Model Loss")
plt.title("Synthetic Testing Data QUAK Space")
plt.legend()
plt.show()